In [6]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.quantinuum import QuantinuumBackend
import matplotlib.pyplot as pl

import os
print(os.getcwd())

from pytket.extensions.qiskit import qiskit_to_tk, AerBackend
from pytket.backends.backendresult import BackendResult

import qiskit as qk

import pickle
import pandas as pd
import numpy as np

/Users/utkarshagrawal/Documents/Postdoc/U_1_exp/Quantinum/Weak measurements


In [8]:
machine = 'H1-1E'
backend = QuantinuumBackend(machine)
backend.login()

print(machine, "status:", backend.device_state(device_name=machine))

H1-1E status: online


In [3]:
def load_file(L,depth,p,Q):
    file_loc = '../../Weak measurements/data/multiple_ancilla/circ_data/special_scrambling/basis_gate_set=1/L='+str(L)+'_depth='+str(depth)+'_Q='+str(Q)+'_p='+str(p)+'_seed=1.imdat'
    with open (file_loc,'rb') as f:
        circ_q = pickle.load(f)
    return circ_q

# This function removes any save_statevector instruction from the qiskit circuit as it is not recognized by quantinuum backend.
def remove_save_statevector(circ_q):

    position = 0
    count = 0
    save_positions = []
    for inst in circ_q.data:
        if inst.operation.name == 'save_statevector':
            count+=1
            save_positions.append(position)
        position+=1

    new_circ = circ_q.copy()
    for pos in reversed(save_positions):
        new_circ.data.pop(pos)

    # print("Before remvoing: # of save instructions = ",count,'\n save positions = ',save_positions,'\n save positions = ',position)

    position = 0
    count = 0
    save_positions = []
    for inst in new_circ.data:
        if inst.operation.name == 'save_statevector':
            count+=1
            save_positions.append(position)
        position+=1
    # print("After remvoing: # of save instructions = ",count,'\n save positions = ',save_positions,'\n save positions = ',position)
    return new_circ


def compiled_tket_circuit(L,depth,p,Q):
    circ_q = load_file(L,depth,p,Q)
    new_circ = remove_save_statevector(circ_q['circuit'])
    circ_p = qiskit_to_tk(new_circ)
    compiled_circ = backend.get_compiled_circuit(circ_p)
    # render_circuit_jupyter(circ_p)

    return compiled_circ
    
def get_L(filename):
    temp = filename[2:4]
    if '_' in temp:
        temp = temp[:-1]
    return int(temp)
    
def get_Q(filename):
    index = filename.index('Q')
    temp = filename[index+2:index+4]
    if '_' in temp:
        temp = temp[:-1]
    return int(temp)

def get_p(filename):
    index = filename.index('p=')
    temp = filename[index+2:index+7]
    while '_' in temp:
        temp = temp[:-1]
    return float(temp)


In [5]:
depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + ''
else:
    depth_label = ''
file_save_loc = 'data/compiled_circuits_multiple_ancillas' + depth_label

if not os.path.isfile(file_save_loc):

    file_dir = file_loc = "../../Weak measurements/data/multiple_ancilla/circ_data/special_scrambling/basis_gate_set=1/"
    compiled_circuits = {}
    for file in os.listdir(file_dir):
        L = get_L(file)
        p = get_p(file)
        Q = get_Q(file)
        depth = int(L*depth_ratio)
        file_loc = file_dir + file
        temp = compiled_tket_circuit(L=L,depth=int(L*depth_ratio),p=p,Q=Q)
        compiled_circuits[(L,depth,p,Q)] = temp

    with open(file_save_loc,'wb') as f:
        pickle.dump(compiled_circuits,f)
